In [ ]:
!pip install datasets sentence-transformers faiss-cpu langchain openai
!pip install sacremoses
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Malikeh1375/medical-question-answering-datasets", name="all-processed", split = "train")

general_qa = [{"question": row["input"], "answer": row["output"]} for row in dataset]

print(general_qa[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

all-processed/train-00000-of-00001-9bfe4(…):   0%|          | 0.00/160M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/246678 [00:00<?, ? examples/s]

{'question': 'Hey Just wondering.  I am a 39 year old female, pretty smallMy heart rate is around 97 to 106 at rest, and my BP is 140/90 and twice I get 175/118I did visit a doctor because I  didnt feel well past month or twoThen the doctor gave me a heart medicine to take the pulse down and BP  (its still in further examination.)But I wondering what it can be? Do I need the medicine really?  Is that bad ?', 'answer': "hello and thank you for using chatbot. i carefully read your question and i understand your concern. i will try to explain you something and give you my opinion. we talk about hypertension if we have mean value that exceeds 140 / 90 mmhg. a person might have high value during emotional and physicals trees so it's mandatory to judge on mean values. usaly hypertension does not give any symptoms but left untreated he slowly modifies the heart. according to heart rhythm, the normal rate is between 50-100 beat for minute. when it exceeds 100 we talk about sinus tachycardia. t

In [ ]:
cardiology_dataset = load_dataset("ilyassacha/cardiology_qa", split="train")
cardiology_qa = [{"question": row["question"], "answer": row["answer"]} for row in cardiology_dataset]
print("Cardiology Q&A samples:", len(cardiology_qa))
print(cardiology_qa[0])

README.md:   0%|          | 0.00/438 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14885 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1654 [00:00<?, ? examples/s]

Cardiology Q&A samples: 14885
{'question': 'My husband of 82 years is on multiple medications for heart disease and Parkensons. His feet are very swollen ...is this because he is retaining fluids and if so what can he do to relieve this. He is under the care of several doctors but they have not offered or explained or shed any light on the cause. margaretmidea@ icloud.com', 'answer': 'hi, thank-you for the brief history.to reduce the retention there should be some muscular activity. it will be active or passive. a physical therapist will play a key role. also keeping the legs elevated will be good for reducing some swelling. use lower limb stockings for compression effect. as due to lack of mobility in the muscular system this is happening. i wish you find a physical therapist who can take the responsibility further and help your husband to the best.  with the grace of god i wish you a speedy recoveryregardsjay in chatbot.'}


In [ ]:
dermatology_dataset = load_dataset("Mreeb/Dermatology-Question-Answer-Dataset-For-Fine-Tuning", split="train")
dermatology_qa = [{"question": row["prompt"], "answer": row["response"]} for row in dermatology_dataset]
print("Dermatology Q&A samples:", len(dermatology_qa))
print(dermatology_qa[0])

README.md:   0%|          | 0.00/929 [00:00<?, ?B/s]

combined_data.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1460 [00:00<?, ? examples/s]

Dermatology Q&A samples: 1460
{'question': 'What is psoriasis and what are its common symptoms?', 'answer': 'Psoriasis is a chronic autoimmune condition that results in the overproduction of skin cells. This overproduction leads to patches of thick, red skin covered with silvery scales. Common symptoms include red patches of skin covered with thick, silvery scales, small scaling spots (commonly seen in children), dry and cracked skin that may bleed, itching, burning, or soreness, thickened, pitted, or ridged nails, and swollen and stiff joints.'}


In [ ]:
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# --- Dataset lists ---
datasets = {
    "dermatology": dermatology_qa,
    "cardiology": cardiology_qa,
    "general": general_qa
}

indexes = {}
mean_embeddings = {}

# Create FAISS index for each dataset
for name, data in datasets.items():
    print(f"Processing {name} dataset with {len(data)} samples...")

    # Encode questions
    embeddings = embedder.encode([item["question"] for item in data], normalize_embeddings=True)

    # Create FAISS index
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))

    # Store index and mean embedding for router
    indexes[name] = index
    mean_embeddings[name] = np.mean(embeddings, axis=0)

    # Optional: Save index
    faiss.write_index(index, f"{name}_vector.index")
    print(f"{name} index saved. Total vectors: {index.ntotal}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Processing dermatology dataset with 1460 samples...
dermatology index saved. Total vectors: 1460
Processing cardiology dataset with 14885 samples...
cardiology index saved. Total vectors: 14885
Processing general dataset with 246678 samples...
general index saved. Total vectors: 246678


In [ ]:
def route_pipeline(question):
    q = question.lower()
    if any(word in q for word in ["skin", "rash", "dermatitis", "eczema", "psoriasis", "acne"]):
        return "dermatology"
    elif any(word in q for word in ["heart", "cardio", "blood pressure", "cholesterol", "arrhythmia", "myocardial"]):
        return "cardiology"
    else:
        return "general"

In [ ]:
def retrieve_docs(question, pipeline, top_k=3):
    q_emb = embedder.encode([question], normalize_embeddings=True).astype("float32")
    D, I = indexes[pipeline].search(q_emb, top_k)
    return [datasets[pipeline][i] for i in I[0]]

In [ ]:
def llm_chain_run(context, question):
    # Dummy LLM response for demonstration
    return f"{context[0 : ]}"

def rag_pipeline(question):
    pipeline = route_pipeline(question)
    retrieved_docs = retrieve_docs(question, pipeline, top_k=3)
    context = " ".join([d["answer"] for d in retrieved_docs])
    final_answer = llm_chain_run(context, question)
    return final_answer

In [ ]:
query = "What are the common causes of neuropathic pain, and how is it treated?"

In [ ]:
print("Query Answer:", rag_pipeline(query))

Query Answer: Neuropathic pain is a type of pain caused by damage or dysfunction of a nerve or of the nervous system. Unlike nociceptive pain, which is caused by tissue damage or inflammation, neuropathic pain is caused by abnormal signaling in the nervous system, which can result in sensations of burning, tingling, numbness, or shooting pain. Neuropathic pain can be caused by a variety of factors, including nerve injuries, infections, autoimmune disorders, vitamin deficiencies, and exposure to toxins. It is a common symptom of many neurological conditions, such as peripheral neuropathy, diabetic neuropathy, postherpetic neuralgia, and trigeminal neuralgia. Treatment for neuropathic pain may involve medications, such as antidepressants, anticonvulsants, or opioids, as well as physical therapy, nerve blocks, or other targeted therapies to manage symptoms and improve quality of life. Neuropathy is a medical condition that affects the peripheral nervous system, and it typically presents w